# Transformer
Bienvenue à l'un des TPs les plus longs de ce repo. Aujourd'hui on va s'intéresser à l'architecture des transformers.
Un transformer est un réseau de neurones qui permet de transformer des séquences de données en d'autres séquences de données. Il est très utilisé dans le domaine du NLP pour faire de la traduction de texte par exemple. C'est aussi l'architecture derrière le fameux ChatGPT, que vous connaissez déjà.

Les modèles de génération d'images comme DALLE utilisent aussi des transformers, pour encoder le texte.

![chatgpt](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/chatgpt.png)

![dalle](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/dalle.png)

## Overview

Personnellement, je trouve que c'est plus facile à comprendre les transformers en commençant par comprendre comment l'utiliser (comprendre les entrées et les sorties) et ensuite en regardant l'architecture.
On va donc commencer par un exemple d'utilisation, puis on va rentrer dans le détail de l'architecture.
Donc pour l'instant, on va considérer le transformer comme un black box.


## Inférence (ou la phase d'utilisation/prédiction)

Pendant l'inférence, le transformer prédit un token (ou un mot, ou un élément de la séquence, c'est la même chose) à la fois. Il prend en compte les tokens précédemment prédits et la séquence d'origine comme entrées pour prédire le token suivant.

Considérons l'exemple de la traduction : le transformer utilise à la fois la phrase source et les mots précédemment prédits de la phrase traduite comme entrées afin de générer le mot suivant de la phrase traduite.

![blackbox](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/blackbox.png)

Pendant la traduction, le transformer prend en entrée la phrase source et la phrase actuellement prédite, puis génère le mot suivant dans la phrase traduite en se basant sur ces deux entrées. Il répète ce processus, en prenant les mots précédemment prédits et la phrase source en entrée à chaque étape, jusqu'à ce que la phrase traduite complète soit générée.

Examinons ça avec un exemple détaillé.

Supposons que le transformer soit déjà entraîné et qu'on veut qu'il traduise "I love oranges." en français. La phrase traduite qu'on veut obtenir est "J'aime les oranges".

La phrase source est `["I", "love", "oranges", "."]`.

Comme c'est le début de l'inférence, la phrase actuellement prédite est vide, mais on ne peut pas mettre une phrase vide dans le transformer, donc on commence par `["<start>"]` comme phrase actuellement prédite.

On a donc ces deux séquences en entrées: `["I", "love", "oranges", "."]` et `["<start>"]`.

On va tokenizer ensuite ces séquences (convertir chaque mot en un nombre le représentant), par exemple `["I", "love", "oranges", "."]` -> `[10, 35, 20, 49]` et `["<start>"]` -> `[40]`.

Le transformer prend ensuite en entrée ces deux séquences, puis génère un vecteur de probabilité sur l'ensemble du vocabulaire, représentant la probabilité de chaque mot possible suivant le token `<start>`.

Puisque le transformer est déjà entraîné, le mot avec la probabilité la plus élevée va être `J'`. On ajoute `J'` à la phrase actuellement prédite.

![first inference](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/probability1.png)

On a donc maintenant `["I", "love", "oranges", "."]` et `["<start>", "J'"]`. Avec la tokenisation, on a `[10, 35, 20, 49]` et `[40, 20]`, en supposant que 20 représente `J'`.
Remarque qu'on a deux tokeniseurs différents pour l'anglais et le français.

Le transformer prend ensuite en entrée ces séquences, puis génère deux vecteurs de probabilité, la première représentant la probabilité du mot possible suivant le token `<start>`, et la seconde celle du mot possible suivant le token `J'`.

![second inference](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/probability2.png)

Remarque que le premier vecteur de probabilité est identique au précédent. C'est parce que le transformer génère toujours les même vecteurs pour les mêmes entrées même s'il y a des tokens supplémentaires dans la phrase actuellement prédite. Les futurs tokens n'influencent pas les vecteurs de probabilité des tokens précédents. C'est parce qu'il y a un masque qu'on appelle `causal mask` qui empêche les tokens de prêter attention aux futurs tokens; on en discutera plus tard dans la section sur l'architecture du transformer.

Puisque on a déjà prédit le mot suivant le token `<start>` à la première étape, on peut ignorer le premier vecteur de probabilité et se concentrer uniquement sur le second.


Comme le transformer est déjà entraîné, le mot avec la probabilité la plus élevée va être "aime".

On obtient donc `["I", "love", "oranges", "."]` et `["<start>", "J'", "aime"]`.

On répète ce processus jusqu'à ce que le transformer génère le token `<end>`, indiquant que la séquence générée est complète. Le transformer est entraîné pour prédire le token `<end>` de manière appropriée pendant la phase d'entraînement, ce qui nous permet de déterminer quand arrêter la boucle de génération.

### Exercice 1: Utilisation d'un transformer pré-entraîné pour faire de la traduction de texte
On va importer un modèle de transformer déjà entraîné et vous allez devoir faire la boucle d'inférence pour traduire la phrase "I love oranges." en français.

Téléchargement des libraries: `pip install transformers sentencepiece`.
Si vous êtes sur colab, alors lancez cette cellule

In [1]:
!pip install transformers sentencepiece

Maintenant on va load le modèle. Vous n'avez pas besoin de comprendre la cellule suivante, tout ce qu'il faut retenir c'est que vous avez un objet `transformer` qui contient le modèle et le tokenizer que vous allez utiliser pour faire l'inférence.

Je vais vous expliquer comment utiliser le modèle et le tokenizer juste après.

In [2]:
!pip install sacremoses
from transformers import MarianMTModel, MarianTokenizer
import torch

class MyTransformer:
    def __init__(self):
        # Load pre-trained model and tokenizer
        model_name = "Helsinki-NLP/opus-mt-en-fr"
        self.tokenizer = MarianTokenizer.from_pretrained(model_name)
        self.model = MarianMTModel.from_pretrained(model_name)
        self.start_token = self.model.config.decoder_start_token_id

    def tokenize(self, text):
        tokenized_input = self.tokenizer(text, return_tensors='pt', truncation=True)
        return tokenized_input["input_ids"]

    def detokenize(self, token_ids):
        return [self.tokenizer._convert_id_to_token(token_id) for token_id in token_ids]

    def forward(self, src_sequence, tgt_sequence):
        """
        src_sequence: torch.LongTensor of shape (batch_size, seq_len) la séquence des ids des mots de la phrase source
        tgt_sequence: torch.LongTensor of shape (batch_size, seq_len) la séquence des ids des mots de la phrase actuellement prédite

        return: torch.FloatTensor of shape (batch_size, seq_len, vocab_size) la probabilité de chaque mot possible pour chaque token de la phrase actuellement prédite
        """
        with torch.no_grad():
            encoder_states = self.model.get_encoder()(src_sequence)
            decoder_states = self.model.get_decoder()(tgt_sequence, encoder_hidden_states=encoder_states.last_hidden_state)
            prediction = self.model.lm_head(decoder_states.last_hidden_state)
            prediction = torch.softmax(prediction, dim=-1)
        return prediction

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

transformer = MyTransformer()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 11.9 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Dans `transformer`, vous avez 2 méthodes que vous pouvez utiliser pour l'inférence: `transformer.tokenize` et `transformer.detokenize` pour convertir les phrases en nombres et vice versa.

Lorsque vous faites ```transformer.tokenize("I love oranges.")```, vous obtenez ```tensor([[47, 1779, 12610, 9, 3, 0]])```,

Et lorsque vous faites ```transformer.detokenize([47, 1779, 12610, 9, 3, 0])```, vous obtenez ```['▁I', '▁love', '▁orange', 's', '.', '</s>']```.

On remarque qu'il y a "\_" devant certains mots. C'est parce que le tokenizer sépare certains mots en sous mots. Par exemple, "love" est devenu "\_love", mais "orange" est séparé en "\_orange" et "s". C'est juste une méthode avancée de faire de la tokenisation, et il y a d'autres façons de faire comme du mot par mot ou du caractère par caractère. ChatGPT par exemple utilise du "Byte Pair Encoding".
Le "\_" représente un espace, donc "\_love" est " love" et "s" est juste "s".

D'ailleurs, `</s>` est le token `<end>`.

Pour utiliser le modèle pré entraîné, vous allez devoir faire `transformer(input_ids, decoder_input_ids)`, où `input_ids` est la séquence tokenisée de la phrase source et `decoder_input_ids` est la séquence tokenisée de la phrase actuellement prédite. `input_ids` et `decoder_input_ids` doivent être des `torch.tensor` de shape `(batch_size, seq_len)` et de type `long`.

Par exemple `transformer.tokenize("I love oranges.")` est un valide `input_ids` car il est de shape `(1, 6)` et de type `long`.

`transformer(input_ids, decoder_input_ids)` va retourner un `torch.tensor` de shape `(batch_size, seq_len, vocab_size)` qui correspond aux vecteurs de probabilité de chaque token de la phrase actuellement prédite. `vocab_size` est le nombre de mots dans le vocabulaire du modèle. Donc pour obtenir le mot suivant, vous allez devoir trouver l'élément avec la plus grande probabilité dans le dernier vecteur de probabilité de `transformer(input_ids, decoder_input_ids)`.

`transformer.start_token` est l'id du token `<start>`. Vous allez devoir l'utiliser pour initialiser `decoder_input_ids` à la première étape de la boucle d'inférence.

In [3]:
print(transformer.tokenize("I love oranges."))
print(transformer.detokenize([47, 1779, 12610, 9, 3, 0]))

tensor([[   47,  1779, 12610,     9,     3,     0]])
['▁I', '▁love', '▁orange', 's', '.', '</s>']


Par exemple, si vous voulez prédire le premier mot de la phrase "I love oranges.", vous allez devoir faire:

In [4]:
input_ids = transformer.tokenize("I love oranges.")
print(input_ids.shape)
decoder_input_ids = torch.tensor([[transformer.start_token]], dtype=torch.long)
print(decoder_input_ids)
print(decoder_input_ids.shape)


prediction = transformer(input_ids, decoder_input_ids)
print(prediction.shape)

id_next_word = torch.argmax(prediction[0, -1]).item()
print(id_next_word)

next_word = transformer.detokenize([id_next_word])
print(next_word)

torch.Size([1, 6])
tensor([[59513]])
torch.Size([1, 1])
torch.Size([1, 1, 59514])
234
['▁J']


### A vous de jouer
Maintenant que vous savez comment utiliser le modèle, essayez de faire la boucle d'inférence pour traduire la phrase "I love oranges." en français en utilisant le modèle.

In [5]:

while next_word != ['</s>'] :
  decoder_input_ids = torch.cat([decoder_input_ids,torch.tensor([[id_next_word]], dtype=torch.long)], dim=1)
  #print(decoder_input_ids.numpy()[0])
  #print(decoder_input_ids.shape)
  #print(transformer.detokenize(decoder_input_ids.numpy()[0]))

  prediction = transformer(input_ids, decoder_input_ids)
  #print(prediction.shape)
  id_next_word = torch.argmax(prediction[0, -1]).item()
  #print(id_next_word)

  next_word = transformer.detokenize([id_next_word])
  print(next_word)

print(prediction.shape)

print(transformer.detokenize(decoder_input_ids.numpy()[0]))

["'"]
['adore']
['▁les']
['▁orange']
['s']
['.']
['</s>']
torch.Size([1, 8, 59514])
['<pad>', '▁J', "'", 'adore', '▁les', '▁orange', 's', '.']


Normalement, vous devriez obtenir "J'adore les oranges.".

## Entraînement
Avant de regarder l'architecture d'un transfo, on va regarder comment on entraîne un transformer. La phase d'entraînement est un peu différente de la phase d'inférence.
En fait, pendant l'entraînement, au lieu de prédire un token à la fois, le transformer va prédire tous les tokens de la target sequence (la phrase qu'on doit prédire) en même temps.

Par exemple, reprenons l'exemple "I love oranges.". La phrase qu'on veut obtenir est "J'aime les oranges.". Au lieu de prédire un mot à la fois, le transformer va prédire `[J', aime, les, oranges, ., <end>]` (enfin, il va essayer puisqu'il n'est pas encore entraîné).

Plus précicément, on va donner au transformer les deux phrases `[I, love, oranges, .]` et `[<start>, J', aime, les, oranges, .]` en `input_ids` et `decoder_input_ids` respectivement, et on veut qu'il:
- prédise `J'` à partir de `[<start>]`
- prédise `aime` à partir de `[<start>, J']`
- prédise `les` à partir de `[<start>, J', aime]`
- prédise `oranges` à partir de `[<start>, J', aime, les]`
- prédise `.` à partir de `[<start>, J', aime, les, oranges]`
- prédise `<end>` à partir de `[<start>, J', aime, les, oranges, .]`

Donc on veut qu'il fasse un peu la même chose qu'en inférence, mais tout parallèlemnt et en un coup, c'est-à-dire prédire directement tous les tokens `[J', aime, les, oranges, ., <end>]` en même temps.

De plus, comme on donne `[<start>, J', aime, les, oranges, .]` à notre transformer, il faudra faire en sorte que le transformer ne puisse pas "tricher" en regardant directement les futurs tokens pour prédire le token actuel. On discutera de ça plus tard dans l'architecture du transformer.


![training phase](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/prediction.png)

Bref, donc notre target sequence (ou labels si vous voulez) est `[J', aime, les, oranges, ., <end>]`. Et on va devoir calculer la loss entre la prédiction du transformer et la target sequence.

Voici un schéma qui illustre l'entraînement du transfo:

![detailed training](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/prediction_6vectors.png)

Remarque que comme la phrase actuellement prédite (ou `decoder_input_ids`, ou `inp_tgt` dans le schéma) contient 6 tokens, alors la prédiction du transformer (ou `out_tgt` dans le schéma) va contenir 6 vecteurs de probabilité aussi, et que notre target sequence (ou `tgt` dans le schéma) contient 6 tokens également.

On va calculer la cross entropy loss entre chaque vecteur de probabilité de la prédiction du transformer et chaque token de la target sequence. Donc on va avoir 6 loss, et on va faire la moyenne de ces 6 loss pour avoir la loss finale.


### Exercice: calculer la cross entropy loss du modèle `transformer` pour la phrase "I love oranges." en français ("J'aime les oranges.")
Sachant que `[234, 6, 17711, 16, 12610, 9, 3, 0]` correspond à la tokenisation de la séquence `[J, ', adore, les, orange, s, ., <end>]` en français.
Calculer la moyenne des 6 cross entropy loss entre la prédiction du transformer et la target sequence.

In [6]:
import torch.nn as nn

target_ids = torch.tensor([234, 6, 17711, 16, 12610, 9, 3, 0])
decoder_input_ids = torch.tensor([[234, 6, 17711, 16, 12610, 9, 3, 0]])
predictions = transformer(input_ids, decoder_input_ids)
predictions = predictions.to(dtype=torch.float)

# Reshaper les prédictions et les cibles pour correspondre à CrossEntropyLoss
predictions = predictions.view(-1, predictions.shape[2]) # (batch_size * seq_len, predictions.shape[2] = vocab_size), (1,8) ici
target_ids = target_ids.view(-1)  # (batch_size * seq_len), (8)  ici

loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(predictions, target_ids)  # CrossEntropyLoss prend en entrée (x,y) avec x les prédictions, et y les targets
#les targets y doivent être des logits (des ids) pour la CrossEntropyLoss et non pas des onehots
print("Loss:", loss.item())

Loss: 10.991252899169922


\Normalement, vous devriez obtenir une loss de 10.526789665222168. Si vous obtenez une différence de plus de 0.01, ce n'est probablement pas normal, il faudra vérifier votre code.

## L'architecture du transformer
Maintenant que vous avez compris les entrées et les sorties du transformer, on va regarder l'architecture du transformer. On va construire le transformer petit à petit en python.

![transformer architecture](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/architecture.png)

Dans le transformer, deux séquences sont données en entrée: Inputs et Outputs (shifted right) (`input_ids` et `decoder_input_ids` dans notre cas).

![input and output shifted right](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/inputs_archi.png)

Dans le cas de la traduction, Inputs est la phrase à traduire, et Outputs (shifted right) est la phrase actuellement prédite.

Par exemple, pour notre cas Inputs est `[I, love, oranges, .]` et Outputs (shifted right) est:
- `[<start>, J', aime, les, oranges, .]` si on est dans la phase d'entraînement
- `[<start>]` si on est au début de la phase d'inférence, et `[<start>, J']` si on a déjà prédit `J'` et qu'on veut prédire `aime` maintenant, etc.


### Inputs embedding

![inputs](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/input_emb_box.png)

La couche inputs embedding sert juste à transformer les tokens en vecteurs de dimension `d_model`. Pour du traitement de texte, on utilise la plupart de temps la couche `nn.Embedding` que vous avez vu dans le TP sur l'introduction à NLP.

Exemple:
![embedding](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/exembedding.png)
Ici, le token 10 est transformé en vecteur `[0.2, -0.5, ..., 1.2, -0.7]`.

Les valeurs de ces vecteurs seront apprises pendant l'entraînement à l'aide de la backpropagation. Durant l'entraînement, le transformer va ajuster ces valeurs pour que les vecteurs représentent les features les plus utiles et informatives des tokens.

Intuitivement, on pourrait imaginer que les mots synonymes auront des vecteurs similaires, et les mots antonymes auront des vecteurs opposés.


#### Similarité

Mais... que'est-ce que ça veut dire "similaires" et "opposés" dans ce contexte? Comment est-ce qu'on peut mesurer la similarité entre deux vecteurs?

Deux méthodes les plus connues sont: la distance euclidienne et la similarité cosinus.

En NLP, on utilise la plupart du temps la similarité cosinus (voire le produit scalaire pour gagner du temps de calcul).

![cosine similarity](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/distance.png)

Une question se pose alors: Pourquoi la similarité cosinus et pas distance euclidienne?

Dans beaucoup de tâches de NLP, la direction dans laquelle un vecteur pointe (son orientation dans l'espace) est plus importante que sa longueur.
La similarité cosinus et le produit scalaire donnent une mesure directe de la similarité en termes de direction de deux vecteurs alors que la distance euclidienne ne mesure que la distance entre deux vecteurs.

Exemple:
- La personne A a acheté 1x oeufs, 1x farine et 1x sucre.
- La personne B a acheté 100x oeufs, 100x farine et 100x sucre
- La personne C a acheté 1x oeufs, 1x Vodka et 1x Red Bull

Par la similarité cosinus, A et B sont plus similaires.
Par la distance euclidienne, A et C sont plus similaires.

Et intuitivement, c'est plus logique de se dire que A et B sont plus similaires que A et C.

Intuitivement, si deux mots sont synonymes, alors leur similarité cosinus sera proche de 1. Si deux mots sont antonymes, alors leur similarité cosinus sera proche de -1 et si deux mots sont complètement différents, alors leur similarité cosinus sera proche de 0, c'est-à-dire les deux vecteurs sont orthogonaux.

#### Implémentation du transformer
Donc pour l'instant, on a ça:

In [7]:
import torch
import torch.nn as nn

class Transformer(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model

    def forward(self, input_ids, decoder_input_ids):
        """
        input_ids: (batch_size, seq_len)
        decoder_input_ids: (batch_size, seq_len)
        """
        inputs = self.embedding(input_ids) # (batch_size, seq_len, d_model)

Notre modèle prend en entrée deux séquences de tokens: `input_ids` et `decoder_input_ids`. On passe `input_ids` (qui est de taille `(batch_size, seq_len)`) dans la couche `self.embedding` pour avoir `inputs` de taille `(batch_size, seq_len, d_model)`.


### Positional encoding

![posenc](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/posenc_box.png)

Le transformer traite tous les tokens en même temps, donc il n'y a pas de notion de position ou d'ordre dans la séquence.

Pourquoi ça pose de problème?

Exemple:
- I love oranges and I hate apples.
- I hate oranges and I love apples.

Ces deux phrases n'ont pas le même sens, mais si on encode pas la position des tokens d'une manière ou d'une autre, pour le transformer, ces deux phrases sont identiques.

L'idée va être de créer un vecteur de position pour chaque token, et d'additionner ces vecteurs de position aux vecteurs de token.

![positional embeddings](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/posenc_add.png)

On appelle ces vecteurs de position "positional embeddings".

#### L'intuition derrière l'addition des positional embeddings
Lorsqu'on additionne deux vecteurs, on déplace le vecteur de départ vers une direction donnée par le vecteur qu'on ajoute. Donc ici, en additionnant le positional embedding au token embedding, on déplace le token embedding vers le cluster/groupe des vecteurs qui représentent les tokens à cette position.

![cluster](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/posenc_intuition.png)

Par exemple, supposons que le premier mot de la phrase est "car". Le token embedding du mot "car" est localisé quelque part dans notre espace de `d_model` dimensions, et lorsque on additionne le positional embedding du premier mot, on déplace ce token embedding vers le cluster des mots qui sont à la première position.

Ce processus va aider le modèle à différencier les mots qui sont à des positions différentes dans la séquence.

#### Comment créer ces positional embeddings?
Une méthode naïve va être de créer le vecteur `[1, 1, 1, ..., 1]` pour le positional embedding de la première position, `[2, 2, 2, ..., 2]` pour la deuxième position, etc.

Mais cette méthode peut donner à des vecteurs de position de valeurs très grandes pour des phrases longues, qui dominent alors les valeurs des vecteurs de token embedding, ce qui rend difficile pour le modèle d'apprendre les features des tokens.

On pourrait tenter de composer ces vecteurs par une fonction bornée comme le sigmoid, mais ça peut poser des problèmes car lorsque les valeurs sont trop grandes, le gradient du sigmoid devient très petit, et donc l'écart entre les vecteurs de position devient très petit, ce qui rend difficile pour le modèle de différencier les tokens à des positions différentes.

![sigmoid](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/gradient_sigmoid.png)

#### Solution des auteurs de l'article sur le transformer
Dans le papier original sur le transformer, les auteurs ont proposé une méthode pour créer les positional embeddings qui est plus efficace que la méthode naïve.

Imagse qui illustrent la méthode:

![pe](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/pe.png)

![posi](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/posi.png)

$PE_{(pos, n)}$ est la valeur du positional embedding à la position $pos$ et à la dimension $n$.

*We trust the author that it works lol* (un jour si j'ai la motivation je ferai une explication détaillée de l'intuition derrière cette méthode)

Je vous file le code pour créer les positional embeddings car ce n'est pas la partie la plus intéressante du transformer.


In [8]:
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()

        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1) # shape = (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe = torch.zeros(max_len, d_model) # shape = (max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)

        # maintenant, on veut faire self.pe = pe
        # mais si on fait ça, lorsqu'on fait model.to("cuda"), self.pe ne sera pas envoyé sur le GPU
        # en utilisant register_buffer, on aura self.pe = pe et self.pe sera envoyé sur le GPU lorsqu'on fait model.to("cuda")
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x.shape = (batch_size, seq_len, d_model)
        seq_len = x.shape[1]
        x = x + self.pe[:seq_len].squeeze()
        return x


Tout ce qu'il faut retenir c'est que la couche `PositionalEncoding` prend en entrée un tenseur de taille `(batch_size, seq_len, d_model)` (ça va être `inputs` pour notre cas) et retourne un tenseur de même taille, mais avec les positional embeddings additionnés aux token embeddings.

#### Exercice: Mettre à jour le code du transformer pour utiliser les positional embeddings

In [9]:
import torch
import torch.nn as nn

class Transformer(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model)
        self.d_model = d_model


    def forward(self, input_ids, decoder_input_ids):
        """
        input_ids: (batch_size, seq_len)
        decoder_input_ids: (batch_size, seq_len)
        """
        inputs = self.embedding(input_ids) # (batch_size, seq_len, d_model)
        pe = self.positional_encoding(inputs)
        return pe

#### Vérification de votre solution
Retournez le résultat du forward, et lancer la cellule suivante. Vérifiez que vous obtenez `tensor([[[[0., 1.]]]])`. (ignorez le `grad_fn`)

In [10]:
transfo = Transformer(1, 2)
x = torch.tensor([[0]])
print(transfo(x, None) - transfo.embedding(x))

tensor([[[0.0000, 1.0000]]], grad_fn=<SubBackward0>)


### Multi-head attention
![mha](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/rectangle_mha_layer.png)

### Single-head Self-Attention

#### Intuition
Considérons la phrase "I love eating orange.". Le mot "orange" a deux sens: la couleur, et le fruit.

Notre but est alors de déterminer le sens du mot "orange" en fonction du contexte. Donc l'idée est de chercher des **key** words dans la phrase qui nous aideront à déterminer le sens du mot "orange" (le **query** word).
Un moyen de faire ça est de calculer la similarité entre le query word et chaque mot de la phrase. Les mots qui ont une forte similarité avec le query word sont les mots qui nous aideront à déterminer le sens du query word.

On va utiliser le produit scalaire pour calculer la similarité.

![dot product](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/orange_dotproduct.png)

Dans l'exemple illustré dans l'image, on voit que les mots "orange" et "eating" ont une forte similarité, avec un score de 1.2. Cela va nous aider à déterminer le sens du mot "orange" dans le contexte.
On remarque d'ailleurs que le score de "orange" avec lui même est de 2, ce qui est normal car le mot "orange" est le query word, donc on s'attend à ce qu'il ait une forte similarité avec lui même.

Bref, une fois qu'on a le score de similarité entre le query et chaque mot, on peut construire un vecteur qui représente le sens du query dans le contexte.
Pour ça, on va faire une somme pondérée de tous les mots de la phrase, où les poids vont être les scores de similarité normalisés. Pour normaliser les scores, on va les passer par la fonction softmax.

En faisant une somme pondérée, on accentue l'influence des mots du contexte qui ont des scores de similarité plus élevés, ce qui encapsule un sens sémantique plus précis pour le query, "orange".

![softmax](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/3dvectorspace_orange.png)

Intuitivement, ce processus va déplacer le vecteur du query word "orange" vers le cluster de mots qui ont un rapport avec les fruits.

#### Détail

Bon ça c'était l'intuition, maintenant on va voir plus en détail comment marche cette couche.

La couche single-head self-attention prend en entrée une séquence de vecteurs générée par notre couche d'embedding et la couche positional encoding.

Pour chaque vecteur $i$ de la séquence, trois vecteurs vont être générés: le vecteur **query**, **key**, et **value** en utilisant 3 couches linéaires. On les notera $Q_i$, $K_i$, $V_i$ respectivement.
Ces vecteurs vont représenter différents aspects de la sémantique du mot dans le contexte de la phrase.

![qkv](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/qkv.png)

Maintenant, pour obtenir le sens contextuel d'un query word $Q_i$, on va calculer la similarité entre $Q_i$ et chaque mot de la séquence en faisant le produit scalaire entre $Q_i$ et chaque vecteur $K_j$.
On obtient alors un vecteur de scores de similarité qu'on va normaliser en passant par la fonction softmax. On note le vecteur de scores normalisés $[\alpha_1, ..., \alpha_n]$.

On peut enfin construire le nouveau vecteur qui représente le sens contextuel du mot $i$ en faisant la somme pondérée des vecteurs $V_j$ avec les poids $[\alpha_1, ..., \alpha_n]$.

![softmax](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/qkv_dotprod.png)

Essayons de comprendre pourquoi on utilise 3 vecteurs Q, K, V à la place d'un seul vecteur.

Considérons la situation où le query word est "network". Dans ce contexte, on voudrait que les mots comme "neural" et "social" aient des scores de similarité élevés, car les phrases "neural network" et "social network" sont courantes. Ça veut dire que le produit scalaire entre le vecteur query et les vecteurs key pour ces mots devrait être élevé.

Donc ça veut dire que les vecteurs keys associés à "neural" et "social" devraient être similaires. Mais comme "neural" et "social" veulent dire des choses différentes, leurs vecteurs values doivent différer considérablement. Si on utilisait un seul vecteur, cette différenciation ne serait pas possible.

C'est pourquoi on utilise 3 vecteurs différents pour représenter chaque mots. Chaque vecteur représente un aspect différent de la sémantique du mot et sert à un but différent.

Revenons à nos moutons. On a donc calculé le nouveau vecteur qui représente le sens contextuel du mot $i$.
Bien sûr, on pourrait itérer ce processus pour chaque $i$ de la séquence, mais on peut aussi paralléliser et accélérer le calcul en utilisant les calculs matriciels pour obtenir tout en un coup.

Notons $Q = [Q_1, ..., Q_n]$, $K = [K_1, ..., K_n]$, $V = [V_1, ..., V_n]$ les matrices qui contiennent les vecteurs query, key, et value de tous les mots de la séquence.

Alors on peut obtenir les scores de similarité entre chaque query word et chaque mot de la séquence en faisant le produit matriciel $$\text{score} = \text{softmax}\left(QK^T\right)$$
Puis on peut obtenir les nouveaux vecteurs en faisant $$V' = \text{score} \cdot V$$

En réalité, avant de faire le softmax pour obtenir le score, on divise par un facteur $\sqrt{d}$ où $d$ est la dimension des vecteurs query, key, et value. Les auteurs du papier ont dit que ça marche mieux donc on va les faire confiance.

On obtient alors une formule compacte: $$V' = \text{softmax}\left(\frac{QK^T}{\sqrt{d}}\right) \cdot V$$

Illustration:
![self attention](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/attention.png)

Tout ce processus s'appelle le self-attention. C'est "self" parce que les query, key, et value sont tous des mots de la même séquence et c'est "attention" parce que le vecteur query "fait attention" aux vecteurs key pour déterminer le vecteur value.

### Multi-head Self-Attention

Dans la couche de single-head self-attention, on applique le processus d'attention une fois sur les données d'entrée. Pour l'attention multi-head, on applique ce processus plusieurs fois (d'où "multi-head"):
Etant donné une séquence d'entrée de vecteurs de taille `(seq_len, d_model)` (si on prend pas en compte la dimension des batches), on divise `d_model` en $h$ parties, chacune de taille `d_model/h`. On obtient alors $h$ séquences de vecteurs de taille `(seq_len, d_model/h)`. On les appelle les *heads*.

Pour chaque head, on a une séquence de vecteurs de taille `(seq_len, d_model/h)`, qui contient un sous-ensemble des features pour chaque mot de la phrase, et on peut appliquer le mécanisme de single-head self-attention à ce sous-ensemble de features.

On obtient alors $h$ nouvelles séquences de vecteurs de taille `(seq_len, d_model/h)`.

L'intuition ici est que chaque head peut se concentrer sur un aspect différent des données d'entrée. En divisant les features entre les heads, on permet à chaque head de se spécialiser dans certains types de relations.
L'exemple suivante n'est pas trop réaliste, mais ça peut aider à construire l'intuition. Par exemple, un head pourrait capturer le genre (masculin, féminin, neutre) d'un nom tandis qu'un autre head pourrait capturer la cardinalité (singulier vs pluriel) d'un nom. Ça pourrait être important pendant la traduction parce que dans beaucoup de langues, le verbe qui doit être utilisé dépend de ces facteurs.

Une fois qu'on a la sortie de chaque head, on doit recombiner ces sorties pour obtenir notre séquence finale de vecteurs, en concaténant les sorties de tous les heads le long de l'axe des features, ce qui donne un séquence finale de vecteurs de taille `(seq_len, d_model)`.

Le split head est illustrée ci-dessous:
![split head](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/splithead.png)

Puis on passe chaque head dans une couche de single-head self-attention:
![mha](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/multihead.png)

![concat](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/concatenatehead.png)


Ensuite, pour projeter la séquence finale dans notre espace de features original, on applique une couche linéaire finale. Les poids de cette couche sont appris pendant l'entraînement, ce qui permet au modèle de déterminer la meilleure façon d'intégrer l'information de tous les heads.

Tout le processus peut être représenté par la formule suivante:
$$\text{MultiHead}(Q, K, V) = \text{Linear}\big(\text{Concat}(\text{head}_1, ..., \text{head}_h)\big)$$

où
$$\text{head}_i= \text{softmax}\left(\frac{Q^{(i)} {K^{(i)}}^T} {\sqrt{d_i}}\right) \cdot V^{(i)}$$

Illustration:
![wholepic](https://raw.githubusercontent.com/uyitroa/draft-transfo-wiki/main/mha-illustration.png)

### Exercice: Implémentation de la couche single-head self-attention
Pour faire produit matriciel entre deux matrices `a` et `b`, vous pouvez utiliser `torch.matmul(a, b)`.

Essayez de compléter la fonction `forward` de la cellule ci-dessous pour implémenter la couche single-head self-attention.

In [11]:
import math
import torch.nn as nn
import torch


class SingleHeadSelfAttention(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model
        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        # x.shape = (batch_size, seq_len, d_model)
        q = self.q_linear(x) # q.shape = (batch_size, seq_len, d_model)
        k = self.k_linear(x)
        v = self.v_linear(x)
        k_transpose = k.transpose(-2, -1) # k est de shape (batch_size, seq_len, d_model), et on veut permuter les deux dernières dimensions d'où -2 et -1
        # k_transpose.shape = (batch_size, d_model, seq_len)

        scores = torch.matmul(self.softmax(torch.matmul(q, k_transpose)/(self.d_model**(1/2))),v) #ici le torch.matmul est essentiel pour le broadcasting

        return scores

### Vérification

Lorsque vous lancez la cellule suivante, vous devriez obtenir
```python
tensor([[[-2.5449, -0.0960],
          [-2.6525, -0.1210]]])
```
comme résultat. (ignorez le `grad_fn`)
Si vous obtenez un résultat différent, vérifiez bien votre implémentation.

In [12]:
import torch
torch.use_deterministic_algorithms(True)
torch.manual_seed(0)

sa = SingleHeadSelfAttention(2)
x = torch.tensor([[[1, 2], [3, 4]]], dtype=torch.float)

print(sa(x))

tensor([[[-2.5449, -0.0960],
         [-2.6525, -0.1210]]], grad_fn=<UnsafeViewBackward0>)


### Exercice: Implémentation de la couche multi-head self-attention
Pour split un tenseur en plusieurs parties, regardez la documentation de `torch.split`: https://pytorch.org/docs/stable/generated/torch.split.html

Pour concaténer plusieurs tenseurs, regardez la documentation de `torch.cat`: https://pytorch.org/docs/stable/generated/torch.cat.html

Vérifiez bien sur quelle dimension vous faites l'opération.
Il faudra utiliser une boucle `for` pour appliquer la self-attention à chaque head.

In [13]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, h):
        super().__init__()
        self.d_model = d_model
        self.h = h
        self.heads = [SingleHeadSelfAttention(d_model//h) for _ in range(h)]
        self.linear = nn.Linear(d_model, d_model)

        # lorsque vous avez une liste de couche, il faut transformer cette liste en un module nn.ModuleList, sinon les paramètres de ces couches ne seront pas enregistrés par PyTorch
        # si vous ne faites pas ça, lorsque vous faites model.to("cuda"), les paramètres de ces couches ne seront pas envoyés sur le GPU automatiquement
        self.heads = nn.ModuleList(self.heads)

    def forward(self, x):
        # x.shape = (batch_size, seq_len, d_model)
        l_x = torch.split(x, split_size_or_sections=self.d_model//self.h, dim=-1)
        x = torch.cat([self.heads[i](l_x[i]) for i in range(self.h)], dim = 2)

        x = self.linear(x)

        return x


### Vérification
Pour vérifier que votre implémentation est correcte, vous pouvez lancer la cellule suivante. Vous devriez obtenir

```python
tensor([[[ 0.3529,  2.5714,  0.9558, -1.3315],
         [ 0.4814,  2.6746,  1.2625, -1.2092]]])
```
comme résultat.

In [14]:
import torch
torch.use_deterministic_algorithms(True)
torch.manual_seed(0)

mha = MultiHeadSelfAttention(4, 2)
x = torch.tensor([[[1, 2, 3, 4], [5, 6, 7, 8]]], dtype=torch.float)

print(mha(x))

torch.Size([1, 2, 4])
tensor([[[ 0.3529,  2.5714,  0.9558, -1.3315],
         [ 0.4814,  2.6746,  1.2625, -1.2092]]], grad_fn=<ViewBackward0>)


### Progrès sur l'implémentation du Transformer
Maintenant, notre classe `Transformer` ressemble à ça:

In [15]:
import torch
import torch.nn as nn

class Transformer(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_enc = PositionalEncoding(d_model)
        self.mha = MultiHeadSelfAttention(d_model, num_heads)
        self.d_model = d_model


    def forward(self, input_ids, decoder_input_ids):
        """
        input_ids: (batch_size, seq_len)
        decoder_input_ids: (batch_size, seq_len)
        """
        inputs = self.embedding(input_ids) # (batch_size, seq_len, d_model)
        inputs = self.pos_enc(inputs) # (batch_size, seq_len, d_model)

        outputs = self.mha(inputs) # (batch_size, seq_len, d_model)

### Feedforward
![feedforward](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/feedforward.png)

La couche feedforward contient simplement deux couches linéaires avec une activation GELU entre les deux. C'est relativement simple comparé au mécanisme d'attention.

Chaque vecteur de la séquence de la sortie de la couche MHA va être transformé par la première couche linéaire, qui projette l'entrée dans un espace de dimension plus élevée, typiquement à une dimension 4 fois plus grande que la dimension originale c'est-à-dire `4*d_model`. C'est pour créer une représentation "plus riche" de chaque mot.

Ensuite, on utilise une deuxième couche linéaire pour transformer les vecteurs de grande dimension vers la dimension originale `d_model`.

![ff](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/ff.png)

### Exercice: Implémentation de la couche feedforward

In [16]:
class FeedForward(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model
        self.linear1 = nn.Linear(d_model, 4*d_model)
        self.linear2 = nn.Linear(4*d_model, d_model)
        self.gelu = nn.GELU()

    def forward(self, x):
        # x.shape = (batch_size, seq_len, d_model)
        x = self.linear1(x)
        x = self.gelu(x)
        x= self.linear2(x)
        return x

### Exercice: Ajouter la couche feedforward au Transformer

In [17]:
class Transformer(nn.Module):
    ...

### Add & Norm
![addnorm](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/addnorm.png)

Cette couche aide à stabiliser les sorties des couches Multi-Head Attention et Feedforward et facilite aussi l'entraînement de notre modèle.

Elle consiste en deux opérations principales:

- Residual Connection (Add): Les sorties de la couche Multi-Head Attention et de la couche Feedforward sont chacune ajoutées à leur entrée originale (avant ces couches), créant ainsi une connexion résiduelle, qui aident à éviter les problèmes de disparition du gradient.

- Layer Normalization (Norm): La connexion résiduelle est suivie d'une couche de normalisation (quoi que, maintenant c'est plus commun de faire la normalisation avant de faire la connection résiduelle). Contrairement à la Batch Normalization, qui normalise sur la dimension du batch, la Layer Normalization s'effectue sur la dimension des features (la dimension `d_model`). Cette normalisation stabilise l'apprentissage du réseau et conduit à une convergence plus rapide et à une meilleure généralisation. La Layer Normalization contient aussi deux paramètres pour ajuster la moyenne et l'écart-type, qui seront appris durant l'entraînement, tout comme la Batch Normalization. La documentation de PyTorch pour la Layer Normalization est [ici](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html).

Gif qui illsutre:
![addnorm gif](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/addnorm.gif)

### Implémentation de la connexion résiduelle et de la LayerNormalization

In [18]:
import torch
import torch.nn as nn

class Transformer(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_enc = PositionalEncoding(d_model)
        self.mha = MultiHeadSelfAttention(d_model, num_heads)
        self.feedforward = FeedForward(d_model)
        self.layer_norm = nn.LayerNorm(d_model)
        self.d_model = d_model


    def forward(self, input_ids, decoder_input_ids):
        """
        input_ids: (batch_size, seq_len)
        decoder_input_ids: (batch_size, seq_len)
        """
        inputs = self.embedding(input_ids) # (batch_size, seq_len, d_model)
        inputs = self.pos_enc(inputs) # (batch_size, seq_len, d_model)

        mha_outputs = self.mha(inputs) # (batch_size, seq_len, d_model)
        mha_outputs = self.layer_norm(mha_outputs + inputs) # (batch_size, seq_len, d_model)

        encoder_outputs = self.feedforward(mha_outputs) # (batch_size, seq_len, d_model)
        encoder_outputs = self.layer_norm(encoder_outputs + mha_outputs) # (batch_size, seq_len, d_model)
        return encoder_outputs

### Plusieurs Encoder Layers
![encoder](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/encoder.png)

En fait, tout ce qu'on a fait jusqu'à maintenant est juste la partie Encodeur du Transformer. Le Transformer est une sorte de Encodeur-Décodeur.
Mais le transformer ne contient pas qu'une seule couche Encodeur, il contient plusieurs couches Encodeur.

![stack](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/stack.png)

Chaque couche Encodeur contient une couche Multi-Head Attention, une couche Feedforward, et une couche Add & Norm. Chaque couche prend en entrée la sortie de la couche précédente.
Cette architecture permet au modèle d'extraire et de traiter des représentations de plus en plus abstraites des données d'entrée, apprenant ainsi des motifs et des relations complexes.

### Implémentation de plusieurs couches Encodeur
Tout d'abord, on va implémenter la couche `EncoderLayer`, et ensuite la couche `TransformerEncoder`.

In [19]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.pos_enc = PositionalEncoding(d_model)
        self.feedforward = FeedForward(d_model)
        self.layer_norm = nn.LayerNorm(d_model)
        self.mha = MultiHeadSelfAttention(d_model, num_heads)
        self.d_model = d_model


    def forward(self, inputs):
        """
        inputs: (batch_size, seq_len, d_model)
        """

        mha_outputs = self.mha(inputs) # (batch_size, seq_len, d_model)
        mha_outputs = self.layer_norm(mha_outputs + inputs) # (batch_size, seq_len, d_model)

        encoder_outputs = self.feedforward(mha_outputs) # (batch_size, seq_len, d_model)
        encoder_outputs = self.layer_norm(encoder_outputs + mha_outputs) # (batch_size, seq_len, d_model)
        return encoder_outputs

### Exercice: Implémentation de TransformerEncoder
Essayez d'implémenter la couche `TransformerEncoder` en utilisant la couche `EncoderLayer` qu'on vient de créer. `TransformerEncoder` prend en entrée un tenseur de taille `(batch_size, seq_len, d_model)` qui correspond à la sortie de la couche Embedding et de la couche PositionalEncoding. Elle doit retourner un tenseur de taille `(batch_size, seq_len, d_model)` qui correspond à la sortie de la dernière couche Encodeur.

In [20]:
class TransformerEncoder(nn.Module):
    def __init__(self, d_model, num_heads, num_layers):
        super().__init__()
        self.layers_list = [EncoderLayer(d_model, num_heads) for _ in range(num_layers)]
        self.d_model = d_model

        self.layers_list = nn.ModuleList(self.layers_list)

    def forward(self, inputs):
        """
        inputs: (batch_size, seq_len, d_model)
        """
        for layer in self.layers_list:
            inputs = layer(inputs)
        return inputs

### Exercice: Modification de la classe `Transformer`
Maintenant, on va modifier la classe `Transformer` pour qu'elle utilise la classe `TransformerEncoder` qu'on vient de créer.

In [21]:
class Transformer(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, num_layers):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_enc = PositionalEncoding(d_model)
        self.encoder = TransformerEncoder(vocab_size, d_model, num_heads, num_layers)
        self.d_model = d_model

    def forward(self, input_ids, decoder_input_ids):
        """
        input_ids: (batch_size, seq_len)
        decoder_input_ids: (batch_size, seq_len)
        """
        inputs = self.embedding(input_ids) # (batch_size, seq_len, d_model)
        inputs = self.pos_enc(inputs) # (batch_size, seq_len, d_model)
        inputs = self.encoder(inputs)
        return inputs

### Décodeur
![decoder](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/decoder.png)

### Exercice: Implémentation de OutputEmbedding et PositionalEncoding
Avant de commencer à regarder le décodeur, il faut implémenter la couche `OutputEmbedding` et la couche `PositionalEncoding` pour le décodeur.
![outputembed](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/outputembed.png)

In [22]:
class Transformer(nn.Module):
    def __init__(self, vocab1_size, vocab2_size, d_model, num_heads, num_layers):
        super().__init__()
        self.input_embedding = nn.Embedding(vocab1_size, d_model)
        self.output_embedding = nn.Embedding(vocab2_size, d_model)
        self.pos_enc = PositionalEncoding(d_model)
        self.encoder = TransformerEncoder(d_model, num_heads, num_layers)

        self.d_model = d_model

    def forward(self, input_ids, decoder_inputs_ids):
        """
        input_ids: (batch_size, seq_len)
        decoder_input_ids: (batch_size, seq_len)
        """
        inputs = self.input_embedding(input_ids)
        inputs = self.pos_enc(inputs)
        inputs = self.encoder(inputs)
        outputs = self.output_embedding(decoder_inputs_ids)
        outputs = self.pos_enc(outputs)
        return inputs

### Multi-Head Self Attention avec Mask
![maskmha](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/maskmha.png)

On a vu que le mécanisme Multi-Head Self Attention joue un rôle essentiel pour mettre en relation les vecteurs dans une séquence. Cependant, lors de la prédiction d'un token pendant la phase d'entraînement, comme on lui donne en entrée toute la phrase cible (target sequence), on ne veut pas que le modèle puisse tricher: on veut s'assurer que la prédiction pour un token particulier ne dépend pas des mots qui viennent après.
Exemple:
- Supponsons qu'on veut prédire le mot "manger" dans la phrase "J'aime manger des oranges.". Comme on a dit précédemment, on veut que le transformer n'utilise que [J', aime] pour prédire "manger". S'il avait accès à toute la phrase "J'aime manger des oranges.", il pourrait juste regarder le mot qui suit "aime" pour prédire.

C'est là qu'intervient le Masked Multi-Head Self Attention.

Le Masked Multi-Head Self Attention est très similaire à Multi-Head Self Attention standard avec juste petite une différence: on additionne un mask sur la matrice des scores d'attention (avant le softmax) pour que les positions futures ne soient pas prises en compte lors du calcul de l'attention.

Illustration:

![causal_mask](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/causal_mask.png)


![causal_mask_after](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/causal_mask_after.png)


On construit ce mask en définissant des valeurs négatives extrêmement grandes (souvent -inf ou -1e9) aux positions pertinentes dans la matrice des scores d'attention, juste avant l'étape softmax, et en mettant des 0 aux autres positions. Ça fait que la fonction softmax produit des 0 pour ces positions, ignorant ainsi efficacement les positions masquées lors du calcul de l'attention. On remarque que si on veut pour que modèle ne puisse pas tricher, on va devoir utiliser un mask qui est une matrice triangulaire avec des `-inf` sur la partie supérieure de la matrice et des 0 sur la partie inférieure. On appelle ce type de mask un **causal mask**:

![matrixcausal](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/matrixcausal.png)

Je vous laisse faire le calcul pour vérifier que le mask empêche bien le modèle de tricher.

![diff](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/diff.png)

![example](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/example.png)

Pour l'exemple ci-dessus, si vous faites les calculs, vous verrez que lors du self-attention du mot "am", le mot "fine" n'est pas pris en compte.

La formule finale pour le masked single-head self attention est la suivante: $$\text{new V} = \text{softmax}\left(\frac{QK^T}{\sqrt{d}} + \text{mask}\right)V$$

Remarque: Bien qu'on a ajouté le mask pour l'entraînement, on ne peut pas l'enlever pendant l'inférence. C'est parce que le modèle est entraîné à utiliser seulement les tokens précédents pour prédire le token suivant. Si on enlève le mask, le modèle va utiliser les tokens futurs pour faire l'attention sur les tokens précédents, ce qui peut donner des résultats incohérents car il n'est jamais entraîné à faire ça.


### Exercice: Implémentation de la classe `MaskedSingleHeadAttention`

In [43]:
import torch.nn as nn
import math


def get_causal_mask(seq_len):
    """
    seq_len: int
    """
    mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1)
    mask = mask.masked_fill(mask==1, float('-inf'))
    return mask


class MaskedSingleHeadAttention(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model

        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        """
        x: (batch_size, seq_len, d_model)
        """
        seq_len = x.shape[1]
        causal_mask = get_causal_mask(seq_len) # (seq_len, seq_len)

        #expanded_mask = causal_mask.unsqueeze(0).expand(x.shape[0], -1, -1)
        #print(expanded_mask.shape)
        # x.shape = (batch_size, seq_len, d_model)
        q = self.q_linear(x) # q.shape = (batch_size, seq_len, d_model)
        k = self.k_linear(x)
        v = self.v_linear(x)
        k_transpose = k.transpose(-2, -1) # k est de shape (batch_size, seq_len, d_model), et on veut permuter les deux dernières dimensions d'où -2 et -1
        # k_transpose.shape = (batch_size, d_model, seq_len)

        scores = torch.matmul(self.softmax((torch.matmul(q, k_transpose)+causal_mask)/(self.d_model**(1/2))),v)

        return scores

### Vérification
Si vous avez bien implémenté la classe `MaskedSingleHeadAttention`, vous devriez obtenir
```python

tensor([[[-1.3326,  0.1852],
         [-2.6525, -0.1210],
         [-4.3539, -0.5156]]])
```

en exécutant la cellule ci-dessous:

In [44]:
import torch

torch.use_deterministic_algorithms(True)
torch.manual_seed(0)

sa = MaskedSingleHeadAttention(2)
x = torch.tensor([[[1, 2], [3, 4], [5, 6]]], dtype=torch.float)


print(sa(x))

tensor([[[-1.3326,  0.1852],
         [-2.6525, -0.1210],
         [-4.3539, -0.5156]]], grad_fn=<UnsafeViewBackward0>)


### Exercice: Implémentation de la classe `MaskedMultiHeadAttention`
La couche `MaskedMultiHeadAttention` ressemble énormément à la couche `MultiHeadAttention` qu'on a implémenté précédemment. La seule différence est qu'on va utiliser la classe `MaskedSingleHeadAttention` qu'on vient d'implémenter au lieu de `SingleHeadAttention`.

In [46]:
class MaskedMultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, h):
        super().__init__()
        self.d_model = d_model
        self.h = h
        self.heads = [MaskedSingleHeadAttention(d_model//h) for _ in range(h)]
        self.linear = nn.Linear(d_model, d_model)

        # lorsque vous avez une liste de couche, il faut transformer cette liste en un module nn.ModuleList, sinon les paramètres de ces couches ne seront pas enregistrés par PyTorch
        # si vous ne faites pas ça, lorsque vous faites model.to("cuda"), les paramètres de ces couches ne seront pas envoyés sur le GPU automatiquement
        self.heads = nn.ModuleList(self.heads)

    def forward(self, x):
        # x.shape = (batch_size, seq_len, d_model)
        l_x = torch.split(x, split_size_or_sections=self.d_model//self.h, dim=-1)
        x = torch.cat([self.heads[i](l_x[i]) for i in range(self.h)], dim = 2)

        x = self.linear(x)

        return x

### Multi-head Cross-Attention
![crossatt](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/crossatt.png)

Le Multi-Head Cross Attention est un mécanisme utilisé pour faire l'attention entre la sortie de l'encodeur et l'entrée du décodeur (plus précisément la sortie de la couche Multi head Self Attention du décodeur).

Dans le Multi-Head Self Attention standard, les queries, keys et values viennent tous du même vecteur. Cependant, dans le Multi-Head Cross Attention, les queries viennent de la couche précédente du décodeur, et les keys et values viennent de la sortie de l'encodeur. Ça permet à chaque token de le décodeur de faire attention à toutes les tokens de la séquence source, d'où "cross-attention".

![illuscross](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/illuscross.png)

Pour de la traduction, on peut imaginer que le décodeur cherche des key words de la séquence source pour savoir quel mot traduire ensuite.
Par exemple, si la séquence source est "I love eating oranges.", lorsqu'on essaie de prédire le mot qui suit `J'`, le modèle va chercher des key words dans la séquence source, et va trouver `I` et `love` qui ont une forte corrélation avec `J'`, et il saura donc que le mot suivant va être le verbe `aimer` et qu'il faut conjuguer à la première personne du singulier.

La formule pour cross-attention est la même que celle pour self-attention, sauf que les queries viennent du décodeur et les keys et values viennent de l'encodeur. On a donc:
$$\text{score}=\text{softmax}\left(\frac{QK^\top}{\sqrt{d}}\right)$$
$$V'=\text{score} \cdot V$$


### Exercice: Implémentation de la classe `SingleHeadCrossAttention`


In [47]:
class SingleHeadCrossAttention(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model

        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.softmax = nn.Softmax(dim=-1)
    def forward(self, decoder_output, encoder_output):
        """
        decoder_output: (batch_size, seq_len_x, d_model)
        encoder_output: (batch_size, seq_len_y, d_model)
        """
        # decoder_output.shape = (batch_size, seq_len, d_model)
        q = self.q_linear(decoder_output) # q.shape = (batch_size, seq_len, d_model)
        k = self.k_linear(encoder_output)
        v = self.v_linear(encoder_output)
        k_transpose = k.transpose(-2, -1) # k est de shape (batch_size, seq_len, d_model), et on veut permuter les deux dernières dimensions d'où -2 et -1
        # k_transpose.shape = (batch_size, d_model, seq_len)

        scores = torch.matmul(self.softmax(torch.matmul(q, k_transpose)/(self.d_model**(1/2))),v)

        return scores

### Vérification
Normalement vous devriez obtenir le résultat suivant:
```python
tensor([[[-4.0098, -0.7054],
         [-4.2934, -0.7711],
         [-4.5612, -0.8332]]])
```
en exécutant la cellule ci-dessous:

In [27]:
import torch

torch.use_deterministic_algorithms(True)
torch.manual_seed(0)

ca = SingleHeadCrossAttention(2)
decoder_output = torch.tensor([[[1, 2], [3, 4], [5, 6]]], dtype=torch.float)
encoder_output = torch.tensor([[[6, 5], [4, 3], [2, 1]]], dtype=torch.float)

print(ca(decoder_output, encoder_output))

tensor([[[-4.0098, -0.7054],
         [-4.2934, -0.7711],
         [-4.5612, -0.8332]]], grad_fn=<UnsafeViewBackward0>)


### Implémentation de la classe `MultiHeadCrossAttention`

In [28]:
class MultiHeadCrossAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads

        self.attention_heads = nn.ModuleList([SingleHeadCrossAttention(d_model//n_heads) for _ in range(n_heads)])
        self.linear = nn.Linear(d_model, d_model)

    def forward(self, decoder_input, encoder_output):
        """
        decoder_output: (batch_size, seq_len_x, d_model)
        encoder_output: (batch_size, seq_len_y, d_model)
        """
        heads = [head(decoder_input, encoder_output) for head in self.attention_heads]
        heads = torch.cat(heads, dim=-1)
        output = self.linear(heads)
        return output

model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

### Overview du décodeur
![decoderr](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/decoderr.png)

On empile aussi les décodeurs comme on a empilé les encodeurs.

### Exercice: Implémentation de la classe `DecoderLayer`

In [48]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.pos_enc = PositionalEncoding(d_model)
        self.layer_norm = nn.LayerNorm(d_model)
        self.mmha = MaskedMultiHeadSelfAttention(d_model, n_heads)
        self.mhca = MultiHeadCrossAttention(d_model, n_heads)
        self.feedforward = FeedForward(d_model)
        self.d_model = d_model

    def forward(self, decoder_output, encoder_output):
        """
        decoder_output: (batch_size, seq_len_x, d_model)
        encoder_output: (batch_size, seq_len_y, d_model)
        """
        mmha_outputs = self.mmha(decoder_output) # (batch_size, seq_len, d_model)
        mmha_outputs = self.layer_norm(mmha_outputs + decoder_output) # (batch_size, seq_len, d_model)

        mhca_outputs = self.mhca(decoder_output, encoder_output)
        mhca_outputs = self.layer_norm(mhca_outputs + mmha_outputs)

        outputs = self.feedforward(mhca_outputs) # (batch_size, seq_len, d_model)
        outputs = self.layer_norm(outputs + mhca_outputs) # (batch_size, seq_len, d_model)
        return outputs


### Exercice: Implémentation de la classe `TransformerDecoder`

In [49]:
class TransformerDecoder(nn.Module):
    def __init__(self, d_model, n_heads, n_layers):
        super().__init__()
        self.layers_list = [DecoderLayer(d_model, n_heads) for _ in range(n_layers)]
        self.d_model = d_model

        self.layers_list = nn.ModuleList(self.layers_list)

    def forward(self, decoder_input, encoder_output):
        """
        decoder_output: (batch_size, seq_len_x, d_model)
        encoder_output: (batch_size, seq_len_y, d_model)
        """
        for layer in self.layers_list:
            decoder_input = layer(decoder_input,encoder_output)
        return decoder_input

### Exercice: Mettre à jour la classe `Transformer`

In [31]:
class Transformer(nn.Module):
    ...

### Presque fini: La dernière couche et le softmax
![lastlayer](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/lastlayer.png)

La dernière couche est juste une couche linéaire + softmax pour prédire les mots suivants. Elle prend en entrée la sortie du décodeur qui est de taille `(batch_size, seq_len, d_model)` et renvoie un tenseur de taille `(batch_size, seq_len, vocab_size)` qui correspond à la probabilité d'avoir chaque mot du vocabulaire à chaque position.

### Implémentation complète de la classe `Transformer`

Pas besoin de softmax dans notre implémentation car il est déjà inclus dans la loss `CrossEntropyLoss` de PyTorch.

In [50]:
class Transformer(nn.Module):
    def __init__(self, vocab1_size, vocab2_size, d_model, n_heads, n_layers):
        super().__init__()
        self.input_embedding = nn.Embedding(vocab1_size, d_model)
        self.output_embedding = nn.Embedding(vocab2_size, d_model)
        self.pos_enc = PositionalEncoding(d_model)

        self.encoder = TransformerEncoder(d_model, n_heads, n_layers)
        self.decoder = TransformerDecoder(d_model, n_heads, n_layers)
        self.last_layer = nn.Linear(d_model, vocab2_size)

    def forward(self, input_ids, decoder_input_ids):
        """
        input_ids: (batch_size, seq_len_x)
        decoder_input_ids: (batch_size, seq_len_y)
        """
        encoder_input = self.input_embedding(input_ids)
        decoder_input = self.output_embedding(decoder_input_ids)

        encoder_input = self.pos_enc(encoder_input)
        decoder_input = self.pos_enc(decoder_input)

        encoder_output = self.encoder(encoder_input)
        decoder_output = self.decoder(decoder_input, encoder_output) # (batch_size, seq_len_y, d_model)
        output = self.last_layer(decoder_output) # (batch_size, seq_len_y, vocab_size)
        return output

### Mdr non je vous ai menti, c'est pas fini
### Key padding mask
En NLP, un problème courant est de travailler avec des phrases de longueurs variables. Certaines phrases peuvent être relativement courtes, mais d'autres peuvent être beaucoup plus longues. Cependant, notre modèle s'attend à ce que toutes les entrées du batch soient de la même longueur.

On va devoir alors utiliser le padding : on ajouter des tokens `<pad>` à toutes les phrases du batch pour qu'elles aient toutes la même longueur.

Ces tokens `<pad>` sont artificiels et ne portent aucune signification sémantique réelle, et donc on ne veut pas que notre modèle prête attention à ces tokens dans la couche Multi head Attention. C'est là que les "key padding mask" vont nous aider.

Un key padding mask est un tenseur binaire de taille `(batch_size, max_seq_len)` qui indique où se trouvent les tokens `<pad>` dans chaque phrase.
Dans ce tenseur, la valeur 1 indique que le token est un `<pad>` et la valeur 0 indique que le token n'est pas un `<pad>`.

Par exemple, considérons une phrase "J'aime manger des oranges", qui a été pad à la longueur 7 : "J'aime manger des oranges \<pad> \<pad> \<pad>". Le key padding mask correspondant serait : `[0 0 0 0 1 1 1]`.

Lors du calcul des scores d'attention, on va utiliser ce mask pour s'assurer que le modèle ne prête pas attention aux tokens de padding. On fait ça en mettant les scores d'attention des paddings à `-inf` avant d'appliquer la fonction softmax.

Le softmax de ces scores sera 0, et donc les poids d'attention pour paddings seront également nuls.

Voici une illustration de la façon dont key padding mask fonctionne dans la couche Multi head attention:

![softmax](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/softmax.png)

Dans l'exemple ci-dessus, on suppose que $K_3$ et $K_4$ sont des tokens de padding. On s'en fiche si $Q_i$ est un token de padding ou non. On ne veut juste pas que $K_3$ et $K_4$ influencent les poids d'attention. On va donc mettre les scores d'attention de $K_3$ et $K_4$ à `-inf` avant d'appliquer le softmax.

Si vous faites le calcul, vous verrez que quand vous calculez $\text{new }V_1$, $\text{new } V_2$ et $\text{new } V_3$ et $\text{new } V_4$, les poids d'attention de $K_3$ et $K_4$ n'infuencent pas la somme pondérée.

Pourquoi on s'en fiche que $Q_i$ soit un padding ou non?
Car les poids d'attention avec $Q_i$ ne sont utilisés que pour calculer $\text{new } V_i$, et pas d'autres $\text{new } V_j$. Vous pouvez faire le calcul pour vérifier.

Dans le self-attention et le cross-attention, si $Q_i$ est un token de padding, alors $V_i$ l'est aussi. Donc on s'en fiche si les poids d'attention avec $Q_i$ influencent $V_i$, car $V_i$ ne porte pas de signification sémantique réelle.

### Implémentation du key padding mask sur `SingleHeadSelfAttention`

In [51]:
class SingleHeadSelfAttention(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model
        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)

    def forward(self, x, key_padding_mask=None):
        """
        Q: (batch_size, seq_len_q, d_model)
        K: (batch_size, seq_len_k, d_model)
        V: (batch_size, seq_len_v, d_model)
        key_padding_mask: (batch_size, seq_len_k)
        """
        Q = self.q_linear(x) # (batch_size, seq_len_q, d_model)
        K = self.k_linear(x) # (batch_size, seq_len_k, d_model)
        V = self.v_linear(x) # (batch_size, seq_len_v, d_model)

        scores = torch.matmul(Q, K.transpose(1, 2)) # (batch_size, seq_len_q, seq_len_k)
        scores = scores / math.sqrt(self.d_model)

        # On met les scores d'attention des paddings à -inf
        if key_padding_mask is not None:
            scores = scores.masked_fill(key_padding_mask.unsqueeze(1) == 1, -1e9)

        weights = torch.softmax(scores, dim=-1) # (batch_size, seq_len_q, seq_len_k)
        output = torch.matmul(weights, V) # (batch_size, seq_len_q, d_model)
        return output

### Exercice: Réimplémentez `MultiHeadSelfAttention`, `SingleHeadCrossAttention`, `MultiHeadCrossAttention` et `Transformer` avec le key padding mask


In [52]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, h):
        super().__init__()
        self.d_model = d_model
        self.h = h
        self.heads = [SingleHeadSelfAttention(d_model//h) for _ in range(h)]
        self.linear = nn.Linear(d_model, d_model)

        # lorsque vous avez une liste de couche, il faut transformer cette liste en un module nn.ModuleList, sinon les paramètres de ces couches ne seront pas enregistrés par PyTorch
        # si vous ne faites pas ça, lorsque vous faites model.to("cuda"), les paramètres de ces couches ne seront pas envoyés sur le GPU automatiquement
        self.heads = nn.ModuleList(self.heads)

    def forward(self, x, key_padding_mask=None):
        # x.shape = (batch_size, seq_len, d_model)
        l_x = torch.split(x, split_size_or_sections=self.d_model//self.h, dim=-1)
        x = torch.cat([self.heads[i](l_x[i], key_padding_mask) for i in range(self.h)], dim = 2)

        x = self.linear(x)

        return x

class SingleHeadCrossAttention(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model

        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, decoder_output, encoder_output, key_padding_mask=None):
        """
        decoder_output: (batch_size, seq_len_x, d_model)
        encoder_output: (batch_size, seq_len_y, d_model)
        """
        # decoder_output.shape = (batch_size, seq_len, d_model)
        q = self.q_linear(decoder_output) # q.shape = (batch_size, seq_len, d_model)
        k = self.k_linear(encoder_output)
        v = self.v_linear(encoder_output)
        k_transpose = k.transpose(-2, -1) # k est de shape (batch_size, seq_len, d_model), et on veut permuter les deux dernières dimensions d'où -2 et -1
        # k_transpose.shape = (batch_size, d_model, seq_len)
        scores = torch.matmul(q, k_transpose)/(self.d_model**(1/2))
        # On met les scores d'attention des paddings à -inf
        if key_padding_mask is not None:
            scores = scores.masked_fill(key_padding_mask.unsqueeze(1) == 1, -1e9)

        scores = torch.matmul(self.softmax(scores),v)

        return scores

class MultiHeadCrossAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads

        self.attention_heads = nn.ModuleList([SingleHeadCrossAttention(d_model//n_heads) for _ in range(n_heads)])
        self.linear = nn.Linear(d_model, d_model)

    def forward(self, decoder_input, encoder_output, key_padding_mask=None):
        """
        decoder_output: (batch_size, seq_len_x, d_model)
        encoder_output: (batch_size, seq_len_y, d_model)
        """
        l_decoder = torch.split(decoder_input, split_size_or_sections=self.d_model//self.n_heads, dim=-1)
        l_encoder = torch.split(encoder_output, split_size_or_sections=self.d_model//self.n_heads, dim=-1)
        heads = [self.attention_heads[i](l_decoder[i], l_encoder[i], key_padding_mask) for i in range(self.n_heads)]
        heads = torch.cat(heads, dim=-1)
        output = self.linear(heads)
        return output

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.pos_enc = PositionalEncoding(d_model)
        self.feedforward = FeedForward(d_model)
        self.layer_norm = nn.LayerNorm(d_model)
        self.mha = MultiHeadSelfAttention(d_model, num_heads)
        self.d_model = d_model


    def forward(self, inputs, src_key_padding_mask = None):
        """
        inputs: (batch_size, seq_len, d_model)
        """

        mha_outputs = self.mha(inputs, src_key_padding_mask) # (batch_size, seq_len, d_model)
        mha_outputs = self.layer_norm(mha_outputs + inputs) # (batch_size, seq_len, d_model)
        encoder_outputs = self.feedforward(mha_outputs) # (batch_size, seq_len, d_model)
        encoder_outputs = self.layer_norm(encoder_outputs + mha_outputs) # (batch_size, seq_len, d_model)
        return encoder_outputs

class TransformerEncoder(nn.Module):
    def __init__(self, d_model, num_heads, num_layers):
        super().__init__()
        self.layers_list = [EncoderLayer(d_model, num_heads) for _ in range(num_layers)]
        self.d_model = d_model

        self.layers_list = nn.ModuleList(self.layers_list)

    def forward(self, inputs, src_key_padding_mask = None):
        """
        inputs: (batch_size, seq_len, d_model)
        """
        for layer in self.layers_list:
            inputs = layer(inputs, src_key_padding_mask)
        return inputs

class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.pos_enc = PositionalEncoding(d_model)
        self.layer_norm = nn.LayerNorm(d_model)
        self.mmha = MaskedMultiHeadSelfAttention(d_model, n_heads)
        self.mhca = MultiHeadCrossAttention(d_model, n_heads)
        self.feedforward = FeedForward(d_model)
        self.d_model = d_model

    def forward(self, decoder_output, encoder_output, src_key_padding_mask = None):
        """
        decoder_output: (batch_size, seq_len_x, d_model)
        encoder_output: (batch_size, seq_len_y, d_model)
        """
        mmha_outputs = self.mmha(decoder_output) # (batch_size, seq_len, d_model)
        mmha_outputs = self.layer_norm(mmha_outputs + decoder_output) # (batch_size, seq_len, d_model)

        mhca_outputs = self.mhca(decoder_output, encoder_output, src_key_padding_mask)
        mhca_outputs = self.layer_norm(mhca_outputs + mmha_outputs)

        outputs = self.feedforward(mhca_outputs) # (batch_size, seq_len, d_model)
        outputs = self.layer_norm(outputs + mhca_outputs) # (batch_size, seq_len, d_model)
        return outputs

class TransformerDecoder(nn.Module):
    def __init__(self, d_model, n_heads, n_layers):
        super().__init__()
        self.layers_list = [DecoderLayer(d_model, n_heads) for _ in range(n_layers)]
        self.d_model = d_model

        self.layers_list = nn.ModuleList(self.layers_list)

    def forward(self, decoder_input, encoder_output, src_key_padding_mask = None):
        """
        decoder_output: (batch_size, seq_len_x, d_model)
        encoder_output: (batch_size, seq_len_y, d_model)
        """
        for layer in self.layers_list:
            decoder_input = layer(decoder_input, encoder_output, src_key_padding_mask)
        return decoder_input

class Transformer(nn.Module):
    def __init__(self, vocab1_size, vocab2_size, d_model, n_heads, n_layers):
        super().__init__()
        self.input_embedding = nn.Embedding(vocab1_size, d_model)
        self.output_embedding = nn.Embedding(vocab2_size, d_model)
        self.pos_enc = PositionalEncoding(d_model)

        self.encoder = TransformerEncoder(d_model, n_heads, n_layers)
        self.decoder = TransformerDecoder(d_model, n_heads, n_layers)
        self.last_layer = nn.Linear(d_model, vocab2_size)

    def forward(self, input_ids, decoder_input_ids, src_key_padding_mask=None):
        """
        input_ids: (batch_size, seq_len_x)
        decoder_input_ids: (batch_size, seq_len_y)
        """
        encoder_input = self.input_embedding(input_ids)
        decoder_input = self.output_embedding(decoder_input_ids)

        encoder_input = self.pos_enc(encoder_input)
        decoder_input = self.pos_enc(decoder_input)

        encoder_output = self.encoder(encoder_input, src_key_padding_mask)
        decoder_output = self.decoder(decoder_input, encoder_output, src_key_padding_mask) # (batch_size, seq_len_y, d_model)
        output = self.last_layer(decoder_output) # (batch_size, seq_len_y, vocab_size)
        return output

### Padding mask dans la loss
Considérons la phrase traduite avec pad: [`<start>`, `J'aime`, `manger`, `des`, `oranges`, `<end>`, `<pad>`, `<pad>`, `<pad>`]

Alors on met en entrée du modèle la phrase sans le dernier padding: [`<start>`, `J'aime`, `manger`, `des`, `oranges`, `<end>`, `<pad>`, `<pad>`] et on veut que le modèle prédise la phrase sans le premier padding: [`J'aime`, `manger`, `des`, `oranges`, `<end>`, `<pad>`, `<pad>`].

Mais si on fait ça, on va forcer le modèle à prédire le token `<pad>` à la fin de la phrase, alors que c'est inutile comme pendant l'inférence, on arrête la boucle dès qu'on obtient le token `<end>`.

Donc on va utiliser un mask de padding pour la loss, qui va masquer les prédictions du modèle pour les tokens de padding.

Supposons qu'on veut que le modèle prédise [`J'aime`, `manger`, `des`, `oranges`, `<end>`, `<pad>`, `<pad>`], alors on va faire la loss entre les prédictions du modèle et la target sequence: on obtient 7 losses: [$\ell_1$, $\ell_2$, $\ell_3$, $\ell_4$, $\ell_5$, $\ell_6$, $\ell_7$].

Avant de faire la moyenne de cette liste, on va enlever les $\ell_6$ et $\ell_7$ car ce sont les losses pour les tokens de padding.

L'idée c'est de multiplier la liste de loss avec [1, 1, 1, 1, 1, 0, 0] pour enlever les losses pour les tokens de padding.

On peut obtenir la liste [1, 1, 1, 1, 1, 0, 0] en cherchant les tokens `<pad>` dans la target sequence.

## Application simple
On va essayer d'entraîner un transformer pour la traduction très simple sur un dataset très simple.

### Dataset
Voici le dataset (mdr)

In [35]:
X_train = ["I like oranges.", "I don't like oranges.", "I like apples.", "I like bananas.", "I don't like pineapples and oranges."]
Y_train = ["J'aime les oranges.", "Je n'aime pas les oranges.", "J'aime les pommes.", "J'aime les bananes.", "Je n'aime pas les ananas et les oranges."]

Pour l'instant on test sur un dataset simple pour vérifier que le modèle marche et apprend.

On verra dans le prochain tp comment générer des histoires.

### A vous de jouer
Il faudra tokenizer puis padder les phrases, puis créer un transformer et l'entraîner.

In [53]:
from collections import defaultdict

# Création de vocabulaires simples
def build_vocab(sentences):
    vocab = defaultdict(lambda: len(vocab))
    vocab["<pad>"] = 0  # Pour le padding
    vocab["<sos>"] = 1  # Début de séquence
    vocab["<eos>"] = 2  # Fin de séquence
    for sentence in sentences:
        for token in sentence.split():
            _ = vocab[token]
    return dict(vocab)

# Tokenisation et conversion en IDs
def tokenize(sentences, vocab):
    tokenized = []
    for sentence in sentences:
        tokens = ["<sos>"] + sentence.split() + ["<eos>"]
        tokenized.append([vocab[token] for token in tokens])
    return tokenized

# Padding pour rendre toutes les séquences de la même longueur
def pad_sequences(sequences, pad_idx=0):
    max_len = max(len(seq) for seq in sequences)
    return [seq + [pad_idx] * (max_len - len(seq)) for seq in sequences]

# Création des vocabulaires
src_vocab = build_vocab(X_train)
tgt_vocab = build_vocab(Y_train)


In [54]:
# Tokenisation
X_train_tokenized = tokenize(X_train, src_vocab)
Y_train_tokenized = tokenize(Y_train, tgt_vocab)

# Padding
X_train_padded = pad_sequences(X_train_tokenized, pad_idx=src_vocab["<pad>"])
Y_train_padded = pad_sequences(Y_train_tokenized, pad_idx=tgt_vocab["<pad>"])

# Conversion en tenseurs
X_train_tensor = torch.tensor(X_train_padded, dtype=torch.long)
Y_train_tensor = torch.tensor(Y_train_padded, dtype=torch.long)


In [55]:
# Paramètres
d_model = 128
n_heads = 8
n_layers = 4
src_vocab_size = len(src_vocab)
tgt_vocab_size = len(tgt_vocab)
# Modèle
model = Transformer(src_vocab_size, tgt_vocab_size, d_model, n_heads, n_layers)

# Fonction de perte et optimiseur
criterion = nn.CrossEntropyLoss(ignore_index=tgt_vocab["<pad>"])  # Ignore les tokens <pad>
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [62]:
from torch.utils.data import DataLoader, TensorDataset

# Dataset et DataLoader
batch_size = 2
dataset = TensorDataset(X_train_tensor, Y_train_tensor)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Boucle d'entraînement
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids, target_ids = batch
        decoder_input_ids = target_ids[:, :-1]  # Décalage (exclure <eos>)
        target_output_ids = target_ids[:, 1:]  # Décalage (exclure <sos>)

        # Réinitialisation des gradients
        optimizer.zero_grad()

        # Prédictions
        predictions = model(input_ids, decoder_input_ids)

        # Reshape pour correspondre à CrossEntropyLoss
        predictions = predictions.reshape(-1, tgt_vocab_size)
        target_output_ids = target_output_ids.reshape(-1)

        # Calcul de la perte
        loss = criterion(predictions, target_output_ids)

        # Backpropagation
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(data_loader):.4f}")


Epoch 1/10, Loss: 0.1951
Epoch 2/10, Loss: 0.1696
Epoch 3/10, Loss: 0.1568
Epoch 4/10, Loss: 0.1040
Epoch 5/10, Loss: 0.0796
Epoch 6/10, Loss: 0.0793
Epoch 7/10, Loss: 0.0691
Epoch 8/10, Loss: 0.0757
Epoch 9/10, Loss: 0.0379
Epoch 10/10, Loss: 0.2377


In [63]:
def translate(sentence, model, src_vocab, tgt_vocab, max_len=50):
    model.eval()
    # Convertir la phrase en IDs
    input_ids = [src_vocab.get(token, 0) for token in sentence.split()]
    input_ids = torch.tensor([[src_vocab["<sos>"]] + input_ids + [src_vocab["<eos>"]]])

    # Initialisation du décodeur
    decoder_input_ids = torch.tensor([[tgt_vocab["<sos>"]]])

    for _ in range(max_len):
        with torch.no_grad():
            predictions = model(input_ids, decoder_input_ids)
            next_token = predictions[:, -1, :].argmax(dim=-1)
            decoder_input_ids = torch.cat([decoder_input_ids, next_token.unsqueeze(0)], dim=1)

            if next_token.item() == tgt_vocab["<eos>"]:
                break

    # Conversion des IDs en texte
    tokens = [key for key, idx in tgt_vocab.items() if idx in decoder_input_ids.squeeze().tolist()]
    return " ".join(tokens[1:-1])  # Exclure <sos> et <eos>


In [64]:
translate("I like oranges .", model, src_vocab, tgt_vocab)

"<eos> J'aime les"

Une fois que vous avez réussi, vous pouvez essayer de l'entraîner sur un vrai dataset: https://pytorch.org/text/main/datasets.html#iwslt2016

## Amélioration de votre score sur le challenge Analyse de sentiment
Vous pouvez essayer d'introduire certaines notions vues dans ce tp pour améliorer votre modèle de classification de sentiments.